Before following this tutorial, please complete the extract data tutorial, we need those files during classification training stage. We have already provided the generated data for the AI City Challenge 2022 dataset.

In [ ]:
# RUN THIS CELL EVERY RESET RUNTIME
WORKDIR='/content/AIC2022-VER'
DATADIR='/content/AIC2022-VER/data' # replace this path
GITHUB_TOKEN=''
WANDB_TOKEN=''

## Clone repo

In [ ]:
!pip install --upgrade --no-cache-dir gdown google-cloud-storage python-Levenshtein webcolors

In [ ]:
REPO_URL=f'https://{GITHUB_TOKEN}@github.com/nhtlongcs/AIC2022-VER/'
!git clone $REPO_URL

Cloning into 'AIC2022-VER'...
remote: Enumerating objects: 2266, done.
remote: Counting objects: 100% (2266/2266), done.
remote: Compressing objects: 100% (1186/1186), done.
remote: Total 2266 (delta 1138), reused 2045 (delta 968), pack-reused 0
Receiving objects: 100% (2266/2266), 12.82 MiB | 18.75 MiB/s, done.
Resolving deltas: 100% (1138/1138), done.


In [ ]:
%cd $WORKDIR
!git checkout main
!git reset --hard HEAD
!git pull

/content/AIC2022-VER
Already on 'main'
Your branch is up to date with 'origin/main'.
HEAD is now at fa8a6a1 Update workflows.yml
Already up to date.


In [ ]:
%cd $WORKDIR
!pip install faiss-gpu
!pip install -r requirements.txt
!pip install -e . 

In [ ]:
import gdown 
def download(token, output: str):
    url = f'https://drive.google.com/uc?id={token}'
    gdown.download(url, output, quiet=False)

In [ ]:
!mkdir -p $DATADIR
%cd $DATADIR

download('1oSFW3ITFPiAVlBrRYctjUANmNid6nob0', 'meta_base.zip')
download('1jMGukcjHcJ9jprvqTBzbzrZPUa10BKVx', 'meta_motion_part.zip')
download('13atYqOhGUEqBD_xpdwOyQqGrkE2psVxX', 'meta_srl_part.zip')

/content/AIC2022-VER/data


Downloading...
From: https://drive.google.com/uc?id=1oSFW3ITFPiAVlBrRYctjUANmNid6nob0
To: /content/AIC2022-VER/data/meta_base.zip
100%|██████████| 14.0G/14.0G [01:58<00:00, 118MB/s]


In [ ]:
!unzip $DATADIR/meta_base.zip         -d $DATADIR/
!unzip $DATADIR/meta_motion_part.zip  -d $DATADIR/meta
!unzip $DATADIR/meta_srl_part.zip     -d $DATADIR/meta

## Training guide


In [ ]:
%%writefile /content/AIC2022-VER/configs/srl/vehtype.yml
global:
  project_name: aic
  username: nhtlong
  name: "vehtype"
  save_dir: ./runs
  use_fp16: true
  debug: false
  verbose: true
  SEED: 1337
  pretrained: null
  resume: null
data:
  name: CropVehDataset
  args:
    CITYFLOW_PATH: "data/meta/extracted_frames"
    SIZE: 288
    CROP_AREA: 1.0 ## new_= CROP_AREA * old_w
    train:
      color_csv: "data/meta/srl/color/col_train.csv"
      vehtype_csv: "data/meta/srl/veh/train_fraction.csv"
      json_path: "data/meta/split/train.json"
      loader:
        batch_size: 32
        num_workers: 2
        shuffle: True
        drop_last: True
    val:
      color_csv: "data/meta/srl/color/col_train.csv"
      vehtype_csv: "data/meta/srl/veh/train_fraction.csv"
      json_path: "data/meta/split/val.json"
      loader:
        batch_size: 32
        num_workers: 2
        shuffle: False
        drop_last: False
extractors:
  img_encoder:
    name: EfficientNetExtractor
    args:
      version: 0
      from_pretrained: True
      freeze: False
model:
  name: VehTypeClassifer
  args:
    NUM_CLASS: 6 # Num of unique vehicles
    EMBED_DIM: 768
metric:
  - name: F1score
    args:
      label_key: "vehtype_lbls"
      num_classes: 6
  - name: Accuracy
    args:
      label_key: "vehtype_lbls"
trainer:
  lr: 0.001
  num_epochs: 100
  clip_grad: 10.0
  evaluate_interval: 1
  print_interval: 20
  save_interval: 1000
callbacks:
  - name: ModelCheckpoint
    args:
      filename: "Vehtype-{epoch}-{val/f1:.4f}"
      monitor: "val/f1"
      verbose: True
      save_top_k: 3
      mode: max
  - name: EarlyStopping
    args:
      monitor: "val/f1"
      min_delta: 0.0001
      patience: 15
      verbose: False
      mode: max
  - name: LearningRateMonitor
    args:
      logging_interval: step

Overwriting /content/AIC2022-VER/configs/srl/vehtype.yml


In [ ]:
!wandb login --relogin $WANDB_TOKEN

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%cd $WORKDIR
!CUDA_VISIBLE_DEVICES=0 python src/train.py \
                        -c configs/srl/color.yml \
                        -o global.save_dir=/content/runs \
                        global.username="hcmus-aic22-ver" \
                        trainer.num_epochs=100 \
                        # trainer.evaluate_interval=5 \

/content/AIC2022-VER
Overriding configurating
Global seed set to 1337
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100% 20.4M/20.4M [00:00<00:00, 47.1MB/s]
Loaded pretrained weights for efficientnet-b0
wandb: Currently logged in as: nhtlong (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.12.13
wandb: Run data is saved locally in /content/runs/wandb/run-20220407_211842-38w7fe9o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run color-2022_04_07-21_18_41
wandb: ⭐️ View project at https://wandb.ai/hcmus-aic22-ver/aic-srl
wandb: 🚀 View run at https://wandb.ai/hcmus-aic22-ver/aic-srl/runs/38w7fe9o
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
1724
data 

In [ ]:
import torch
from opt import Opts

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from src.models import MODEL_REGISTRY

from pathlib import Path


def check(cfg, pretrained_ckpt=None):
    model = MODEL_REGISTRY.get(cfg.model["name"])(cfg)
    model = model.load_from_checkpoint(pretrained_ckpt, config = cfg, strict=True)
    trainer = pl.Trainer(
        gpus=-1 if torch.cuda.device_count() else None,  # Use all gpus available
        accelerator="ddp" if torch.cuda.device_count() > 1 else None,
        sync_batchnorm=True if torch.cuda.device_count() > 1 else False,
    )
    trainer.validate(model)
    del trainer
    del cfg
    del model

pretrained_ckpt = "/content/runs/aic-srl/38w7fe9o/checkpoints/Color-epoch=96-val/f1=0.9451.ckpt"
cfg_path = '/content/AIC2022-VER/configs/srl/color.yml'

cfg = Opts(cfg=cfg_path).parse_args([])
check(
    cfg,
    pretrained_ckpt=pretrained_ckpt
)

Overriding configurating
Loaded pretrained weights for efficientnet-b0


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/AIC2022-VER/lightning_logs


Loaded pretrained weights for efficientnet-b0
1724
data load
431
data load


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

f1: 0.93950 | 

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val/f1             0.9394972406377078
        val/loss            0.02804606594145298
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
